In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
train_data = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test_data = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
train_data.head()

In [ ]:
# Checking target column 
np.unique(train_data.target, return_counts=True)

In [ ]:
# Visualizing data distribution
target_count = train_data.groupby('target').size().reset_index(name='counts')
plt.bar(target_count.target, target_count.counts)
plt.xticks([0,1], labels=["Not disaster tweets", "disaster tweets"])
plt.title("Target Distribution")
plt.show()

# Data Cleaning and Preprocessing

In [ ]:
def preprocess(reviews):
    tokenizer = RegexpTokenizer(r'\w+')
    review = str(reviews)
    review = review.lower()
    review = review.replace("<br /><br />", "")
    tokens = tokenizer.tokenize(review)

    stop_words = set(stopwords.words('english'))
    stopwords_removed = [i for i in tokens if i not in stop_words]

    ps = PorterStemmer()
    stem_text = [ps.stem(i) for i in stopwords_removed]

    cleaned_reviews = " ".join(stem_text)

    return cleaned_reviews

In [ ]:
# CLeaning our training Data
X = train_data.text.apply(preprocess)
y = train_data.target

# Splitting the Data into Train and Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X), y, test_size=0.20, random_state=10)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

# Creating Model Pipeline

In [ ]:
model1 = Pipeline([
    ('tfidf', TfidfVectorizer()), 
    ('mnb', MultinomialNB())
])
model2 = Pipeline([
    ('c_vec', CountVectorizer()), 
    ('mnb', MultinomialNB())
])
model3 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('bern', BernoulliNB())
])
model4 = Pipeline([
    ('c_vec', CountVectorizer()),
    ('bern', BernoulliNB())
])

In [ ]:
# Fitting and training the model now
model1.fit(X_train,y_train)
model2.fit(X_train,y_train)
model3.fit(X_train,y_train)
model4.fit(X_train,y_train)

In [ ]:
print('Model 1 Train score', model1.score(X_train,y_train))
print('Model 1 Test score', model1.score(X_test,y_test))
print()
print('Model 2 Train score', model2.score(X_train,y_train))
print('Model 2 Test score', model2.score(X_test,y_test))
print()
print('Model 3 Train score', model3.score(X_train,y_train))
print('Model 3 Test score', model3.score(X_test,y_test))
print()
print('Model 4 Train score', model4.score(X_train,y_train))
print('Model 4 Test score', model4.score(X_test,y_test))

# Visualizing the accuracy from the models

In [ ]:
models = {'Model 1': model1.score(X_test,y_test),
          'Model 2': model2.score(X_test,y_test),
          'Model 3': model3.score(X_test,y_test),
          'Model 4': model4.score(X_test,y_test)
         }
keys = [i for i in models.keys()]
#print(keys)
values = [i*100 for i in models.values()]
#print(values)
plt.figure(figsize=(15,5))
plt.barh(keys, values)
  
for index, value in enumerate(values):
    plt.text(value, index,
             str("%.2f"%value))
plt.title("Model Accuracy")
plt.show()

### We can conclude that Model 3 and Model 4 are giving the best accuracy of 79.19%

# Predicting on Test data

In [ ]:
# Cleaning the test data
test_set = test_data.text.apply(preprocess)

In [ ]:
pred = model3.predict(test_set)

In [ ]:
# Creating a new DataFrame for submission
submission_df = pd.DataFrame(list(zip(test_data.iloc[:,0],pred)), columns=['id', 'target'])

In [ ]:
submission_df.to_csv('submission.csv', index=False)